In [1]:
import gc
from collections import Counter
from functools import partial

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, log_loss
from sklearn.ensemble import RandomForestClassifier

In [2]:
train_df = pd.read_csv("data/ds-ai-spring-hack/train.csv")
test_df = pd.read_csv("data/ds-ai-spring-hack/test.csv")

/home/dima/python/competitions/promua/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_df.head()

,comment_id,company_id,user_id,product_id,comment,rating,date_created,user_date_created,user_ip,user_agent,is_fake
0,1,2169053,91933,6.0,компания не выполнила заказ без обьяснения пр...,1,2014-10-23 20:26:44,2010-02-03 21:03:22,178.92.70.220,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,False
1,2,709306,91933,2.0,"Впервые встретился такой професиональный, отве...",5,2016-12-10 08:38:27,2010-02-03 21:03:22,94.179.59.162,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,False
2,3,35287,91933,NaN,"Несерйозная, некоректная менеджер інтернет маг...",1,2017-02-11 15:55:14,2010-02-03 21:03:22,37.73.137.188,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False
3,4,1778322,91933,1.0,"Не рекомендую. Не отвечают, не выполняют заказы.",1,2017-06-26 18:00:45,2010-02-03 21:03:22,37.73.149.180,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False
4,5,2821533,91933,NaN,"На письма не отвечают, заказы не выполняют. Не...",1,2017-07-06 16:57:40,2010-02-03 21:03:22,37.73.221.126,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False


In [4]:
train_df.shape

(1281499, 11)

In [5]:
train_df.columns

Index(['comment_id', 'company_id', 'user_id', 'product_id', 'comment',
       'rating', 'date_created', 'user_date_created', 'user_ip', 'user_agent',
       'is_fake'],
      dtype='object')

In [6]:
test_df.head()

,comment,comment_id,company_id,date_created,month_created,product_id,rating,user_agent,user_date_created,user_id,user_ip
0,"спасибо все отлично, качество как на картинке,...",1655822,2214161,2018-03-19 21:36:07,2018-03,1301549.0,5,NaN,2017-11-01 00:00:46,12969242,NaN
1,Заказывала шоколад Риттер Спорт 10 шт. К сожал...,1655823,2333068,2017-12-10 13:46:49,2017-12,699700.0,5,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2017-11-01 00:02:19,12969248,91.90.13.210
2,"Быстро отзвонились, охотно отвечали на все воп...",1655824,752692,2017-11-01 00:07:38,2017-11,NaN,5,Mozilla/5.0 (Windows NT 6.1; rv:38.0) Gecko/20...,2017-11-01 00:02:44,12969251,93.115.95.204
3,Очень оперативный и внимательный к покупателю ...,1655825,334751,2017-11-20 23:40:01,2017-11,1279247.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
4,Все прошло очень оперативно! Заказ привезли в ...,1655826,2263650,2017-11-20 23:39:51,2017-11,1225545.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28


In [7]:
test_df.shape

(252099, 11)

In [8]:
Counter(train_df.company_id).most_common(10)

[(2648571, 2901),
 (42440, 2850),
 (1892310, 2768),
 (277907, 2755),
 (1980780, 2649),
 (1280427, 2316),
 (893220, 2121),
 (210989, 1985),
 (364050, 1894),
 (1731900, 1871)]

In [9]:
def create_time_features(df):
    
    comment_registration_delta = pd.to_datetime(df.date_created) - pd.to_datetime(df.user_date_created)
    comment_registration_delta = comment_registration_delta.astype("timedelta64[D]")
    df["registration_comment_delta_days"] = np.log(comment_registration_delta)
    
    hour = pd.to_datetime(df.date_created).dt.hour
    
    df["hour_sin"] = np.sin(2 * hour * np.pi / 24)
    df["hour_cos"] = np.cos(2 * hour * np.pi / 24)
    
    return df

In [10]:
train_df = create_time_features(train_df)
test_df = create_time_features(test_df)

/home/dima/python/competitions/promua/env/lib/python3.5/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


0          20
1           8
2          15
3          18
4          16
5          21
6          12
7          13
8          21
9          16
10         14
11          8
12         14
13          9
14         23
15         11
16         10
17         11
18         18
19         11
20         20
21         19
22         11
23         23
24          2
25         13
26         12
27         19
28         14
29         13
           ..
1281469    12
1281470    14
1281471    14
1281472    14
1281473    15
1281474    21
1281475    16
1281476    16
1281477    16
1281478    16
1281479    18
1281480    18
1281481    19
1281482    18
1281483    19
1281484    22
1281485    22
1281486    22
1281487    22
1281488    22
1281489    22
1281490    22
1281491    22
1281492    22
1281493    22
1281494    22
1281495    22
1281496    22
1281497    23
1281498    22
Name: date_created, Length: 1281499, dtype: int64
0         21
1         13
2          0
3         23
4         23
5         23
6         23
7    

In [11]:
def create_ip_features(df):
    
    def _ip_class(ip):
        
        try:
            ip0, ip1, ip2, ip3 = map(int, ip.split("."))
            
            # 1.0.0.1 to 126.255.255.254
            if 126 >= ip0 >= 1 and 255 >= ip1 >= 0 and 255 >= ip2 >= 0 and 254 >= ip3 >= 1:
                return 0
            # 128.1.0.1 to 191.255.255.254
            elif 191 >= ip0 >= 128 and 255 >= ip1 >= 1 and 255 >= ip2 >= 0 and 254 >= ip3 >= 1:
                return 1
            # 192.0.1.1 to 223.255.254.254
            elif 223 >= ip0 >= 192 and 255 >= ip1 >= 1 and 254 >= ip2 >= 0 and 254 >= ip3 >= 1:
                return 2
            # 224.0.0.0 to 239.255.255.255
            elif 239 >= ip0 >= 224 and 255 >= ip1 >= 0 and 255 >= ip2 >= 0 and 255 >= ip3 >= 0:
                return 3
            # 240.0.0.0 to 254.255.255.254
            elif 254 >= ip0 >= 240 and 255 >= ip1 >= 0 and 255 >= ip2 >= 0 and 254 >= ip3 >= 0:
                return 4
            else:
                return 5
        except:
            return 6
        
    df["ip_class"] = df.user_ip.astype(str).apply(_ip_class)
    
    return df

In [12]:
def create_text_features(df):
    
    df["comment_length"] = df.comment.apply(len)
    return df

In [13]:
train_df = create_text_features(train_df)
test_df = create_text_features(test_df)

In [ ]:
count_vectorizer = CountVectorizer(min_df=0.01, max_df=0.2)
train_comment_vectorized = count_vectorizer.fit_transform(train_df.comment.astype("str"))
test_comment_vectorized = count_vectorizer.transform(test_df.comment.astype("str"))

In [ ]:
count_vectorizer = CountVectorizer(min_df=0.05)
train_agent_vectorized = count_vectorizer.fit_transform(train_df.user_agent.astype("str"))
test_agent_vectorized = count_vectorizer.transform(test_df.user_agent.astype("str"))

In [ ]:
y_train = train_df.is_fake

In [ ]:
categorical_columns = [
    "rating"
]

In [ ]:
to_drop = [
    "comment", "comment_id", "product_id", "date_created",
    "user_date_created", "user_ip", "user_agent", "user_id", "company_id"
]

In [ ]:
X_train = pd.concat(
    [
        train_df.drop(to_drop + ["is_fake"], axis=1),
        pd.DataFrame(train_comment_vectorized.todense()),
        pd.DataFrame(train_agent_vectorized.todense())
    ],
    axis=1
)
    
X_test = pd.concat(
    [
        test_df.drop(to_drop + ["month_created"], axis=1),
        pd.DataFrame(test_comment_vectorized.todense()),
        pd.DataFrame(test_agent_vectorized.todense())
    ],
    axis=1
)

In [ ]:
del train_df, train_comment_vectorized, train_agent_vectorized
del test_df, test_comment_vectorized, test_agent_vectorized

In [ ]:
gc.collect()

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_data_train = lgb.Dataset(
    X_train,
    label=y_train, 
    free_raw_data=False
)

In [ ]:
def lgb_f1_score(preds, dtrain):
    labels = dtrain.get_label()
    preds = preds > 0.5
    
    return "f1_score", f1_score(labels, preds), True

def lgb_binary_loss(preds, dtrain):
    labels = dtrain.get_label()
    
    return "binary_loss", log_loss(labels, preds), False


In [ ]:
params = {
    "objective": "binary", 
    "metric": "binary_logloss", 
    "learning_rate": 0.03, 
    "random_state": 42,
    "scale_pos_weight": 5,
    "depth": 3,
    "num_leaves": 6
}

In [ ]:
history = lgb.cv(
    params,
    lgb_data_train,
    nfold=5, 
    num_boost_round=100000,
    early_stopping_rounds=50, 
    verbose_eval=10,
    categorical_feature=categorical_columns,
    feval=lgb_f1_score
)

In [ ]:
%%time
gbm = lgb.train(
    params,
    lgb_data_train,
    num_boost_round=480,
    valid_sets=[lgb_data_train],
    feval=lgb_f1_score,
    verbose_eval=10
) 

In [ ]:
submission = pd.read_csv("data/ds-ai-spring-hack/sample_submission.csv")
new_test_df = pd.read_csv("data/ds-ai-spring-hack/test.csv")

In [ ]:
test_ids = list(new_test_df.comment_id)

In [ ]:
test_prediction = gbm.predict(X_test)

In [ ]:
test_prediction_clipped = np.clip(test_prediction, 0.1, 0.9)

In [ ]:
test_prediction_binary = test_prediction > 0.5

In [ ]:
print(test_prediction_binary.sum())

In [ ]:
for column, importance in zip(X_train.columns, gbm.feature_importance()):
    print(column, importance)

In [ ]:
submission_file = pd.DataFrame(
    {
        "comment_id": test_ids,
        "is_fake": test_prediction_clipped
    }
)

In [ ]:
submission_file.to_csv("dima_v15_clipped.csv", index=False)